In [ ]:
#https://www.dataquest.io/blog/tutorial-text-classification-in-python-using-spacy/

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

import string
import spacy
spacy.load('en_core_web_sm')
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

#from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [2]:
df = pd.read_excel('/Users/Suwani/Desktop/Moodys Project/Cleaned data/To analyze/Only_P_N.xlsx')

In [3]:
df.head()

,Headline,my_label
0,Sri lanka halfway between a flawed democracy a...,-1
1,Five connected to a Sri Lankan drug ring arres...,1
2,“Sri Lanka would function better without polit...,1
3,US Navy to bring Rs. 25 million to Sri Lanka’s...,1
4,A FREAK WANNABE DEMOCRAT CANNOT WORK IN SRI LA...,-1


In [4]:
df = df.replace(-1,0)

In [5]:
# Create list of punctuation marks
punctuations = string.punctuation

In [6]:
# Create list of stopwords
nlp = spacy.load("en_core_web_sm")
stop_words = spacy.lang.en.stop_words.STOP_WORDS


In [7]:
# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [8]:
# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

In [9]:
# BoW converts text into the matrix of occurrence of words within a given document
#ngram_range parameter in the code below sets the lower and upper bounds of the our ngrams
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))

In [10]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [11]:

X = df['Headline'] # the features we want to analyze
ylabels = df['my_label'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.3)

In [12]:
# Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

# Create pipeline using Bag of Words
pipe = Pipeline([("cleaner", predictors()), #clean and preprocess the text
                 ('vectorizer', bow_vector), # to create the bag of words matrix for the text
                 ('classifier', classifier)]) #performs the logistic regression to classify the sentiments

# model generation
pipe.fit(X_train,y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
         steps=[('cleaner', <__main__.predictors object at 0x1a268ac710>),
                ('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 tok...u)\\b\\w\\w+\\b',
                                 tokenizer=<function spacy_tokenizer at 0x1a24b74378>,
                                 vocabulary=None)),
                ('classifier',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
            

In [13]:
#from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(X_test)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.7951388888888888
Logistic Regression Precision: 0.7964912280701755
Logistic Regression Recall: 0.9956140350877193
